## Download section 4, 5, 7, 8

#### this script combines the webscraping scripts for sections 4,5,7,8 for the ultrasoundcases.info site 


In [ ]:
import os
import pandas as pd
import urllib.request
import numpy as np 
import matplotlib.pyplot as plt
import cv2 as cv
import requests
from bs4 import BeautifulSoup
import urllib.request
from urllib.error import HTTPError


## 1. Web scrape + download

### 1A. Section 4 web scraping + download at the same time

In [ ]:
# handle request and return as parsed HTML
def request_data(url):
    """
    Request data from HMSS website and parse with beautifulsoup.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

# download one image at a time
def download_imgs(urllink, tgt_path, filename):
    try: 
        # full path: tgt_path + filename
        full_tgt_path = os.path.join(tgt_path, filename)
        # download image to the folder path
        urllib.request.urlretrieve(urllink, full_tgt_path)
        print('{} saved to {}.'.format(filename, full_tgt_path))
        return True
    except HTTPError as e: # exception for the http error (404 not found page)
        print(f"Error downloading image from {urllink}: {e}") 
        return False

In [ ]:
#col_names = ['group', 'subgroup', 'subgroup url', 'case description', 'img url', 'img name', 'case id/cover img id', 'crop r1', 'crop r2', 'crop col1', 'crop col2', 'tumor type', 'Notes']
col_names = ['group', 'subgroup', 'subgroup url', 'case description', 'img url', 'img name', 'sex', 'age', 'body part']

data_4 = pd.DataFrame(columns=col_names)

In [ ]:
"""
Scraping the data from HMSS website for section 4 (4.1-4.5) 
"""

base_url = 'https://www.ultrasoundcases.info/'
INIT_URL = "https://www.ultrasoundcases.info/cases/head-and-neck/"
init_soup = request_data(INIT_URL)
first_level = init_soup.find_all("div", {"class": "candidate-filter"})
tgt_path = 'imgs/'
# make sure file directory exists
if not os.path.exists(tgt_path):
    os.makedirs(tgt_path)

# go through all first level categories (first level = body system (EX. 4 Head and Neck))
for item_l1 in first_level:
    # get the title from h4>a
    item_l1_title = item_l1.find("h4").find("a").text
    # only fetch section 4 items
    if (item_l1_title != 'Head and Neck'):
        continue
    print('Fetching', item_l1_title)
    
    # get the second level categories
    second_level = item_l1.find("ul").find_all("li")
    
    # iterate through each item in the second level (second level = group of body system (EX. 4.1 Thyroid Gland))
    for item_l2 in second_level:
        item_l2_title = item_l2.find("a").text
        item_l2_url_title = item_l2.find("a")['href']
        item_l2_id = item_l2.find("a")["data-id"] 

        print("Fetching", item_l2_title, item_l2_id)

        third_level = requests.get(
            "https://www.ultrasoundcases.info/api/cases/list/" + str(item_l2_id)
        ).json()
        
        # iterate through each item in the third level (third level = subgroup of group (EX. 4.1.1 Thyroid Congenital Abnormalities))
        for item_l3 in third_level: 
            item_l3_title = item_l3["title"]
            item_l3_urltitle = item_l3["urltitle"]
            item_l3_id = item_l3["id"]

            #print("Fetching", item_l3_title, item_l3_id)

            fourth_level = requests.post(
                "https://www.ultrasoundcases.info/api/cases/list/" +  str(item_l3_id) + '/',
                data={"type": "subsubcat"}
            ).json()

            # iterate through each item in the fourth level (fourth level = specific cases of subgroup (EX. Hypoplasia of the left thyroid lobe))
            for item_l4 in fourth_level:
                item_l4_id = item_l4['id']
                item_l4_title = item_l4['title']
                item_l4_subtitle = item_l4['subtitle']
                item_l4_thumb = item_l4['thumb']
                item_l4_urltitle = item_l4['urltitle']

                #print("Fetching", item_l4_title, item_l4_id, item_l4_urltitle)

                item_l5_soup = request_data(
                    "https://www.ultrasoundcases.info/{}/".format(item_l4_urltitle)
                )

                # img information
                fifth_level = item_l5_soup.find("div", {"class": "portfolio"})
                fifth_level = fifth_level.find_all("img")

                # patient details
                l5_details = item_l5_soup.find("div", {"class": "information"})
                l5_h4 = l5_details.find("h4")
                # check if details about case exist
                if(l5_h4.text == 'Details'):
                    l5_patient_info = l5_details.find_all("li")
                    for i, li in enumerate(l5_patient_info):
                        details = ''.join(li.find_all(string=True, recursive=False)).strip()
                        if i == 0:
                            l5_sex = details
                        if i == 1:
                            l5_age = details
                        if i == 2:
                            l5_body_part = details
                else:
                    l5_sex = np.nan
                    l5_age = np.nan
                    l5_body_part = np.nan
                    

                # iterate through each item in the case level (fifth level = image details for the specific case)
                for item_l5 in fifth_level:  # case level
                    item_l5_img_url_src = item_l5["src"]
                    item_l5_img_url = "https://www.ultrasoundcases.info" + item_l5_img_url_src

                    img_soup = request_data(
                        item_l5_img_url
                    )

                    #print("Fetching", item_l5_img_url)

                    # prep data for dataframe
                    item_l3_url = base_url + item_l2_url_title + item_l3_urltitle
                    img_name = item_l5_img_url.split('/')
                    item_l5_img_name = img_name[-1]

                    if not (item_l5_img_name.endswith(".jpg")):
                        # not a jpg image, so do not add it, go to the next image
                        continue

                    # download images here
                    if (download_imgs(item_l5_img_url, tgt_path, item_l5_img_name)):
                        # if an image is downloaded
                        # create row for dataframe
                        data_4.loc[len(data_4.index)] = [item_l2_title, item_l3_title, item_l3_url, item_l4_subtitle, item_l5_img_url, item_l5_img_name, l5_sex, l5_age, l5_body_part]



In [ ]:
# write dataframe to csv file
data_4.to_csv('section4/section4.csv', index=False)

### 1B. Section 5 web scraping + download

In [ ]:
data_5 = pd.DataFrame(columns=col_names)

In [ ]:
"""
Scraping the data from HMSS website for section 5 (5.1-5.8)
"""

base_url = 'https://www.ultrasoundcases.info/'
INIT_URL = "https://www.ultrasoundcases.info/cases/breast-and-axilla/"
init_soup = request_data(INIT_URL)
first_level = init_soup.find_all("div", {"class": "candidate-filter"})
tgt_path = 'imgs/'
# make sure file directory exists
if not os.path.exists(tgt_path):
    os.makedirs(tgt_path)

# go through all first level categories (first level = body system (EX. 5 Breast and Axilla))
for item_l1 in first_level:
    # get the title from h4>a
    item_l1_title = item_l1.find("h4").find("a").text
    # only fetch section 4 items
    if (item_l1_title != 'Breast and Axilla'):
        continue
    print('Fetching', item_l1_title)
    
    # get the second level categories
    second_level = item_l1.find("ul").find_all("li")
    
    # iterate through each item in the second level (second level = group of body system (EX. 5.1 Benign Lesions))
    for item_l2 in second_level:
        item_l2_title = item_l2.find("a").text
        item_l2_url_title = item_l2.find("a")['href']
        item_l2_id = item_l2.find("a")["data-id"] 

        print("Fetching", item_l2_title, item_l2_id)

        third_level = requests.get(
            "https://www.ultrasoundcases.info/api/cases/list/" + str(item_l2_id)
        ).json()
        
        # iterate through each item in the third level (third level = subgroup of group (EX. 5.1.1 Benign cysts with non mass lesions))
        for item_l3 in third_level: 
            item_l3_title = item_l3["title"]
            item_l3_urltitle = item_l3["urltitle"]
            item_l3_id = item_l3["id"]

            print("Fetching", item_l3_title, item_l3_id)

            fourth_level = requests.post(
                "https://www.ultrasoundcases.info/api/cases/list/" +  str(item_l3_id) + '/',
                data={"type": "subsubcat"}
            ).json()

            # iterate through each item in the fourth level (fourth level = specific cases of subgroup (EX. Benign cysts with non mass lesions))
            for item_l4 in fourth_level:
                item_l4_id = item_l4['id']
                item_l4_title = item_l4['title']
                item_l4_subtitle = item_l4['subtitle']
                item_l4_thumb = item_l4['thumb']
                item_l4_urltitle = item_l4['urltitle']

                print("Fetching", item_l4_title, item_l4_id, item_l4_urltitle)

                item_l5_soup = request_data(
                    "https://www.ultrasoundcases.info/{}/".format(item_l4_urltitle)
                )

                # img information
                fifth_level = item_l5_soup.find("div", {"class": "portfolio"})
                fifth_level = fifth_level.find_all("img")

                # patient details
                l5_details = item_l5_soup.find("div", {"class": "information"})
                l5_h4 = l5_details.find("h4")
                
                # check if details about case exist
                if(l5_h4.text == 'Details'):
                    l5_patient_info = l5_details.find_all("li")
                    for i, li in enumerate(l5_patient_info):
                        details = ''.join(li.find_all(string=True, recursive=False)).strip()
                        if i == 0:
                            l5_sex = details
                        if i == 1:
                            l5_age = details
                        if i == 2:
                            l5_body_part = details
                else:
                    l5_sex = np.nan
                    l5_age = np.nan
                    l5_body_part = np.nan
                    

                # iterate through each item in the case level (fifth level = image details for the specific case)
                for item_l5 in fifth_level:  # case level
                    item_l5_img_url_src = item_l5["src"]
                    item_l5_img_url = "https://www.ultrasoundcases.info" + item_l5_img_url_src

                    img_soup = request_data(
                        item_l5_img_url
                    )

                    print("Fetching", item_l5_img_url)

                    # prep data for dataframe
                    item_l3_url = base_url + item_l2_url_title + item_l3_urltitle
                    img_name = item_l5_img_url.split('/')
                    item_l5_img_name = img_name[-1]

                    if not (item_l5_img_name.endswith(".jpg")):
                        # not a jpg image, so do not add it, go to the next image
                        continue

                    # download images here
                    if (download_imgs(item_l5_img_url, tgt_path, item_l5_img_name)):
                        # if an image is downloaded
                        # create row for dataframe
                        data_5.loc[len(data_5.index)] = [item_l2_title, item_l3_title, item_l3_url, item_l4_subtitle, item_l5_img_url, item_l5_img_name, l5_sex, l5_age, l5_body_part]


In [ ]:
# write dataframe to csv file
data_5.to_csv('section5/section5.csv', index=False)

### 1C. Section 7 web scraping + download

In [ ]:
data_7 = pd.DataFrame(columns=col_names)

In [ ]:
"""
Scraping the data from HMSS website for section 7 (7.2-7.5)
"""

base_url = 'https://www.ultrasoundcases.info/'
INIT_URL = "https://www.ultrasoundcases.info/cases/musculo-skeletal-bone-muscle-nerves-and-other-soft-tissues/"
init_soup = request_data(INIT_URL)
first_level = init_soup.find_all("div", {"class": "candidate-filter"})
tgt_path = 'imgs/'
# make sure file directory exists
if not os.path.exists(tgt_path):
    os.makedirs(tgt_path)

# go through all first level categories (first level = body system (EX. 7 Musculoskeletal, bone, muscle, nerves and other soft tissues))
for item_l1 in first_level:
    # get the title from h4>a
    item_l1_title = item_l1.find("h4").find("a").text
    # only fetch section 4 items
    if (item_l1_title != 'Musculoskeletal, bone, muscle, nerves and other soft tissues'):
        continue
    print('Fetching', item_l1_title)
    
    # get the second level categories
    second_level = item_l1.find("ul").find_all("li")
    
    # iterate through each item in the second level (second level = group of body system (EX. 7.2 Muscle))
    for item_l2 in second_level:
        item_l2_title = item_l2.find("a").text
        item_l2_url_title = item_l2.find("a")['href']
        item_l2_id = item_l2.find("a")["data-id"] 
        
        # skip section 7.1
        if item_l2_title == '7.1 Bone':
            continue

        print("Fetching", item_l2_title, item_l2_id)

        third_level = requests.get(
            "https://www.ultrasoundcases.info/api/cases/list/" + str(item_l2_id)
        ).json()
        
        # iterate through each item in the third level (third level = subgroup of group (EX. 7.2.1 Muscle ruptures lower extremity upper leg))
        for item_l3 in third_level: 
            item_l3_title = item_l3["title"]
            item_l3_urltitle = item_l3["urltitle"]
            item_l3_id = item_l3["id"]

            print("Fetching", item_l3_title, item_l3_id)

            fourth_level = requests.post(
                "https://www.ultrasoundcases.info/api/cases/list/" +  str(item_l3_id) + '/',
                data={"type": "subsubcat"}
            ).json()

            # iterate through each item in the fourth level (fourth level = specific cases of subgroup (EX. Lower extremety: upper leg))
            for item_l4 in fourth_level:
                item_l4_id = item_l4['id']
                item_l4_title = item_l4['title']
                item_l4_subtitle = item_l4['subtitle']
                item_l4_thumb = item_l4['thumb']
                item_l4_urltitle = item_l4['urltitle']

                print("Fetching", item_l4_title, item_l4_id, item_l4_urltitle)

                item_l5_soup = request_data(
                    "https://www.ultrasoundcases.info/{}/".format(item_l4_urltitle)
                )

                # img information
                fifth_level = item_l5_soup.find("div", {"class": "portfolio"})
                fifth_level = fifth_level.find_all("img")

                # patient details
                l5_details = item_l5_soup.find("div", {"class": "information"})
                l5_h4 = l5_details.find("h4")

                # check if details about case exist
                if(l5_h4.text == 'Details'):
                    l5_patient_info = l5_details.find_all("li")
                    for i, li in enumerate(l5_patient_info):
                        details = ''.join(li.find_all(string=True, recursive=False)).strip()
                        if i == 0:
                            l5_sex = details
                        if i == 1:
                            l5_age = details
                        if i == 2:
                            l5_body_part = details
                else:
                    l5_sex = np.nan
                    l5_age = np.nan
                    l5_body_part = np.nan
                    

                # iterate through each item in the case level (fifth level = image details for the specific case)
                for item_l5 in fifth_level:  # case level
                    item_l5_img_url_src = item_l5["src"]
                    item_l5_img_url = "https://www.ultrasoundcases.info" + item_l5_img_url_src

                    img_soup = request_data(
                        item_l5_img_url
                    )

                    print("Fetching", item_l5_img_url)

                    # prep data for dataframe
                    item_l3_url = base_url + item_l2_url_title + item_l3_urltitle
                    img_name = item_l5_img_url.split('/')
                    item_l5_img_name = img_name[-1]

                    if not (item_l5_img_name.endswith(".jpg")):
                        # not a jpg image, so do not add it, go to the next image
                        continue

                    # download images here
                    if (download_imgs(item_l5_img_url, tgt_path, item_l5_img_name)):
                        # if an image is downloaded
                        # create row for dataframe
                        data_7.loc[len(data_7.index)] = [item_l2_title, item_l3_title, item_l3_url, item_l4_subtitle, item_l5_img_url, item_l5_img_name, l5_sex, l5_age, l5_body_part]



In [ ]:
# write dataframe to csv file
data_7.to_csv('section7/section7.csv', index=False)

### 1D. Section 8 web scraping + download

In [ ]:
data_8 = pd.DataFrame(columns=col_names)

In [ ]:
"""
Scraping the data from HMSS website for section 8 (8.1-8.4)
"""

base_url = 'https://www.ultrasoundcases.info/'
INIT_URL = "https://www.ultrasoundcases.info/cases/thorax/"
init_soup = request_data(INIT_URL)
first_level = init_soup.find_all("div", {"class": "candidate-filter"})
tgt_path = 'imgs/'
# make sure file directory exists
if not os.path.exists(tgt_path):
    os.makedirs(tgt_path)

# go through all first level categories (first level = body system (EX. 8 Thorax))
for item_l1 in first_level:
    # get the title from h4>a
    item_l1_title = item_l1.find("h4").find("a").text
    # only fetch section 4 items
    if (item_l1_title != 'Thorax'):
        continue
    print('Fetching', item_l1_title)
    
    # get the second level categories
    second_level = item_l1.find("ul").find_all("li")
    
    # iterate through each item in the second level (second level = group of body system (EX. 8.1 Pulmonary pathology))
    for item_l2 in second_level:
        item_l2_title = item_l2.find("a").text
        item_l2_url_title = item_l2.find("a")['href']
        item_l2_id = item_l2.find("a")["data-id"] 

        print("Fetching", item_l2_title, item_l2_id)

        third_level = requests.get(
            "https://www.ultrasoundcases.info/api/cases/list/" + str(item_l2_id)
        ).json()
        
        # iterate through each item in the third level (third level = subgroup of group (EX. 8.1.1 Pneumonia and air space consolidation))
        for item_l3 in third_level: 
            item_l3_title = item_l3["title"]
            item_l3_urltitle = item_l3["urltitle"]
            item_l3_id = item_l3["id"]

            print("Fetching", item_l3_title, item_l3_id)

            fourth_level = requests.post(
                "https://www.ultrasoundcases.info/api/cases/list/" +  str(item_l3_id) + '/',
                data={"type": "subsubcat"}
            ).json()

            # iterate through each item in the fourth level (fourth level = specific cases of subgroup (EX. Pneumonia and air space consolidation))
            for item_l4 in fourth_level:
                item_l4_id = item_l4['id']
                item_l4_title = item_l4['title']
                item_l4_subtitle = item_l4['subtitle']
                item_l4_thumb = item_l4['thumb']
                item_l4_urltitle = item_l4['urltitle']

                print("Fetching", item_l4_title, item_l4_id, item_l4_urltitle)

                item_l5_soup = request_data(
                    "https://www.ultrasoundcases.info/{}/".format(item_l4_urltitle)
                )

                # img information
                fifth_level = item_l5_soup.find("div", {"class": "portfolio"})
                fifth_level = fifth_level.find_all("img")

                # patient details
                l5_details = item_l5_soup.find("div", {"class": "information"})
                l5_h4 = l5_details.find("h4")
                
                # check if details about case exist
                if(l5_h4.text == 'Details'):
                    l5_patient_info = l5_details.find_all("li")
                    for i, li in enumerate(l5_patient_info):
                        details = ''.join(li.find_all(string=True, recursive=False)).strip()
                        if i == 0:
                            l5_sex = details
                        if i == 1:
                            l5_age = details
                        if i == 2:
                            l5_body_part = details
                else:
                    l5_sex = np.nan
                    l5_age = np.nan
                    l5_body_part = np.nan
                    

                # iterate through each item in the case level (fifth level = image details for the specific case)
                for item_l5 in fifth_level:  # case level
                    item_l5_img_url_src = item_l5["src"]
                    item_l5_img_url = "https://www.ultrasoundcases.info" + item_l5_img_url_src

                    img_soup = request_data(
                        item_l5_img_url
                    )

                    print("Fetching", item_l5_img_url)

                    # prep data for dataframe
                    item_l3_url = base_url + item_l2_url_title + item_l3_urltitle
                    img_name = item_l5_img_url.split('/')
                    item_l5_img_name = img_name[-1]
                    
                    if not (item_l5_img_name.endswith(".jpg")):
                        # not a jpg image, so do not add it, go to the next image
                        continue


                    # download images here
                    if (download_imgs(item_l5_img_url, tgt_path, item_l5_img_name)): 
                        # if an image is downloaded
                        # create row for dataframe
                        data_8.loc[len(data_8.index)] = [item_l2_title, item_l3_title, item_l3_url, item_l4_subtitle, item_l5_img_url, item_l5_img_name, l5_sex, l5_age, l5_body_part]


In [ ]:
print(item_l5_img_url)

In [ ]:
# write dataframe to csv file
data_8.to_csv('section8/section8.csv', index=False)

### 1: combine .csv files

In [ ]:
# section 4 csv
section4 = 'section4/section4.csv'
df_4 = pd.read_csv(section4)
#df_4.head()

# section 5 csv
section5 = 'section5/section5.csv'
df_5 = pd.read_csv(section5)

# section 7 csv
section7 = 'section7/section7.csv'
df_7 = pd.read_csv(section7)

# section 8 csv
section8 = 'section8/section8.csv'
df_8 = pd.read_csv(section8)

In [ ]:
# combine the dataframes to one dataframe
df_all = [df_4, df_5, df_7, df_8]
df = pd.DataFrame()
df = pd.concat(df_all, ignore_index=True)

In [ ]:
df

### 2: download images

In [ ]:
# prepare image folders and load image urls from HMSS.csv
tgt_path = 'imgs/'

if not os.path.exists(tgt_path):
    os.makedirs(tgt_path)

urllinkimage = df['img url']

img_names = df['img name']

In [ ]:
# download images
def download_imgs(urllink, tgt_path, names):
    for i, img_url in enumerate(urllink.values ):
        filename = '{}'.format(names[i])
        full_tgt_path = '{}{}'.format(tgt_path, filename)
        print(img_url)
        urllib.request.urlretrieve(img_url, full_tgt_path)
        
        print('{} - {} saved to {}.'.format(i, filename, full_tgt_path))

In [ ]:
download_imgs(urllinkimage, tgt_path, img_names)